In [1]:
import pandas as pd
from pymongo import MongoClient
import json
import numpy as np
import geopandas
import sys
import matplotlib.pyplot as plt
import os
import requests
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap

In [2]:
def connectMongoDb(): 
    dbName = "datamad0620"
    mongodbURL = f"mongodb://localhost/{dbName}"

    client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
    db = client.get_database()
connectMongoDb()

In [3]:
dbName = "datamad0620"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()
    
    
def companiesMongo_count(country_code):
    query = {"offices.country_code":f"{country_code}"}
    
    res = list(db.companies.find(query))
    return len(res) 

companiesMongo_count("SGP")

84

In [32]:
def companiesMongo(country_code):
    query = {"offices.country_code":f"{country_code}"}
    
    projection ={"offices":1, "name":1, "founded_year":1,"total_money_raised":1, "category_code":1}
    res = list(db.companies.find(query,projection))
    return res

result= companiesMongo("SGP")
r=pd.DataFrame(result).sort_values(by="category_code", ascending=False).head(100)
r.groupby('category_code').agg({'category_code':"count"})


,category_code
category_code,
advertising,5
analytics,1
biotech,1
cleantech,1
consulting,3
ecommerce,2
enterprise,7
finance,1
games_video,2


In [27]:
def limipiezaData(companies,code):
    df = pd.DataFrame(list(companies))
    offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
    clean_offices = pd.concat([df[["name","_id"]], offices], axis=1)
    clean_offices = clean_offices.rename(columns={"_id":"company_id"})
    filtered = clean_offices[(clean_offices['country_code']== f"{code}")]
    return filtered   

data = limipiezaData(result, "SGP")

In [28]:
import numpy as np

def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
    

data["geopoint"] = data.apply(transformToGeoPoint, axis=1)

In [29]:
data=data.dropna(subset=['longitude'])
data=data.dropna(subset=['latitude'])


In [30]:
m = Map(location=[1.290270,103.851959],zoom_start=14)
for i, row in data.iterrows():
    Marker(
        location=row[["latitude","longitude"]],
        tooltip=row["name"],
        popup=row["address1"]
    ).add_to(m)
m

In [31]:
def requestFoursquare(query,radio):
    
    client_s = os.getenv("CLIENT_SECRET")
    client_idd = os.getenv("4S_CLIENT_ID")
    lat = 1.298370
    long = 103.891290

    params = {"client_id": client_idd,
              "client_secret":"LNVYSPVLUTJITTBVZ1DHWMWISC43K43N1DTACC5CYWFYRRCZ",
              "v": "20180323",
              "ll":f'{lat},{long}',
              "radius":f'{radio}',
              "query":f'{query}', 
              "limit":100    }

    url = f"https://api.foursquare.com/v2/venues/explore"
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    print(url)
    return data["response"]

requestFoursquare("restaurant",3000)

https://api.foursquare.com/v2/venues/explore


{}

### ponderacion

In [35]:
total = 87
training_w=87/total
school_w=26/total
party_w=50/total
starbucks_w=9/total
vegan=1/total
basketbal=1/total
dog=1/total

- Traning: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- Work-life balance: 30% of the company have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot
- All people in the company have between 25 and 40 years, give them some place to go to party.
- The CEO is Vegan
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog "Pepe" needs a hairdresser every month. Ensure there's one not too far away


In [ ]:
def requestFoursquare(query,radio):
    client_idd = os.getenv("4S_CLIENT_ID")
    client_secret=os.getenv("CLIENT_SECRET")
    lat = 1.298370
    long = 103.891290

    params = {"client_id": client_idd,
              "client_secret":client_secret,
              "v": "20180323",
              "ll":f'{lat},{long}',
              "radius":f'{radio}',
              "query":f'{query}', 
              "limit":20 }

    url = f"https://api.foursquare.com/v2/venues/explore"
    
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    
    return data
requestFoursquare("coffe",2000)

In [ ]:
basketball_list = []
for e in basketball_stadium["response"]["groups"]:
    for i in e["items"]:
        name=i['venue']["name"]
        latitud=i['venue']["location"]["lat"]
        longitud=i['venue']["location"]["lng"]
        basketball_list.append({"Requirement":"Court","name":name,"latitude":latitud,"longitude":longitud,"location":{"type":"Point","coordinates":[longitud,latitud]}})

In [59]:
import googlemaps
from dotenv import load_dotenv
load_dotenv()

def gPlaces (coord,rad=1000,query):
    apiKey = os.getenv('GPLACES_APIKEY')

    gmaps =googlemaps.Client(key=api)
    
    places = gmaps.places_nearby(location=f"{coord}", radius=rad,open_now=False, type=query)
    return places["results"]



SyntaxError: non-default argument follows default argument (<ipython-input-59-85407f1940f0>, line 5)

In [48]:
gPlaces("1.298370,103.891290",1000,"coffe")
results_map=[(places[i]["name"], places[i]["geometry"]["location"]["lat"], places[i]["geometry"]["location"]["lng"]) for i in range(len(places))]
results_map=pd.DataFrame(results_map, columns=['name', 'latitude', 'longitude'])
results_map

,name,latitude,longitude
0,Teo Heng KTV Studio,1.303761,103.900896
1,Blu Jaz Cafe,1.300610,103.858948
2,Teo Heng KTV Studio,1.295191,103.859720
3,Basement Entertainment,1.302899,103.902834
4,Grains and spices,1.304133,103.903745
5,Cash Studio Family Karaoke,1.318544,103.893824
6,K-Net Music Pte Ltd,1.302908,103.864945
7,K Box,1.323634,103.877451
8,Hawaii Cabaret & Nite-club,1.303869,103.861465
9,K.STAR Karaoke @ Suntec City,1.295711,103.858649


In [50]:
from shapely.geometry import Point, Polygon
import geopandas as gpd
import descartes
%matplotlib inline

geometry=[Point(xy) for xy in zip(results_map["latitude"], results_map["longitude"])]
geo_df=gpd.GeoDataFrame(results_map,geometry=geometry)
geo_df

,name,latitude,longitude,geometry
0,Teo Heng KTV Studio,1.303761,103.900896,POINT (1.304 103.901)
1,Blu Jaz Cafe,1.300610,103.858948,POINT (1.301 103.859)
2,Teo Heng KTV Studio,1.295191,103.859720,POINT (1.295 103.860)
3,Basement Entertainment,1.302899,103.902834,POINT (1.303 103.903)
4,Grains and spices,1.304133,103.903745,POINT (1.304 103.904)
5,Cash Studio Family Karaoke,1.318544,103.893824,POINT (1.319 103.894)
6,K-Net Music Pte Ltd,1.302908,103.864945,POINT (1.303 103.865)
7,K Box,1.323634,103.877451,POINT (1.324 103.877)
8,Hawaii Cabaret & Nite-club,1.303869,103.861465,POINT (1.304 103.861)
9,K.STAR Karaoke @ Suntec City,1.295711,103.858649,POINT (1.296 103.859)


In [52]:
geo_df=geo_df.dropna(subset=['longitude'])
geo_df=geo_df.dropna(subset=['latitude'])

In [53]:
m = Map(location=[1.290270,103.851959],zoom_start=14)
for i, row in geo_df.iterrows():
    icon = Icon(
                color="red",
                prefix="fa",
                icon="trophy"
            )
    Marker(
        location=row[["latitude","longitude"]],
        tooltip=row["name"],
    ).add_to(m)
m